In [36]:
#! pip install jsonrpcclient

from jsonrpcclient import request, parse, Ok
import logging
import requests
import pandas as pd
import numpy as np


In [82]:
url = "https://muddy-falling-river.solana-mainnet.quiknode.pro/f6caf7f878f08e5b5b1d88aa3eff9238693e525a/"
pg_num = 1
search = True  # sam thinks this a good idea ....
creater_slug = "3xVDoLaecZwXXtN59o6T3Gfxwjcgf8Hc9RfoqBn995P9"
all_results = []

while search:
    response = requests.post(
        url=url,
        json=request(
            "qn_fetchNFTsByCreator",
            [
                {
                    "creator": creater_slug,
                    "page": pg_num,
                    "perPage": 10000,
                }
            ],
        ),
    )
    parsed = parse(response.json())
    pg_num += 1

    # print(len(parsed.result["assets"]))

    if pg_num > int(parsed.result["totalPages"]):
        search = False
    if isinstance(parsed, Ok) and len(parsed.result["assets"]) >= 1:
        print("yay")
        all_results.append(parsed.result["assets"])
    else:
        logging.error(parsed.message)
        search = False
    print(f"page ={pg_num}")

page =2
yay
page =3
yay
page =4
yay
page =5
yay
page =6
yay
page =7
yay
page =8
yay
page =9
yay
page =10
yay
page =11
yay
page =12
yay
page =13
yay
page =14
yay
page =15
yay
page =16
yay
page =17
yay
page =18
yay
page =19
yay
page =20
yay
page =21
yay
page =22
yay
page =23
yay
page =24
yay
page =25
yay
page =26
yay
page =27
yay
page =28
yay
page =29
yay
page =30
yay
page =31
yay
page =32
yay
page =33
yay
page =34
yay
page =35
yay
page =36
yay
page =37
yay
page =38
yay
page =39
yay
page =40
yay
page =41
yay
page =42
yay
page =43
yay
page =44
yay
page =45
yay
page =46
yay
page =47
yay
page =48
yay
page =49
yay
page =50
yay
page =51
yay
page =52
yay
page =53
yay
page =54
yay
page =55
yay
page =56
yay
page =57
yay
page =58
yay
page =59
yay
page =60
yay
page =61
yay
page =62
yay
page =63
yay
page =64
yay
page =65
yay
page =66
yay
page =67
yay
page =68
yay
page =69
yay
page =70
yay
page =71
yay
page =72
yay
page =73
yay
page =74
yay
page =75
yay
page =76
yay
page =77
yay
page =78
yay
page =7

In [146]:
result = []
for sublist in all_results:
    for item in sublist:
        result.append(item)
df_all = pd.DataFrame.from_records(result)
df_traits = pd.json_normalize(result, "traits", record_prefix="traits_", meta="name")

In [147]:
df_traits.head(2)

,traits_trait_type,traits_value,name
0,Background,Green,Okay Bear #3507
1,Fur,Black,Okay Bear #3507


In [148]:
df_all.head(2)

,name,collectionName,tokenAddress,collectionAddress,imageUrl,traits,chain,creators,network,description,provenance
0,Okay Bear #3507,Okay Bears,HgGVXqmZDdd2taSTkRLqSGdGKqXVfXWXxdC2BwXTzRnx,7ttWYWpc9H9NUQnEaH64qWLwXcHMYwFBu1WrtwjyAHC3,https://bafybeigsln4ka7vedwfidzi6rhm3vdsvs32cj...,"[{'trait_type': 'Background', 'value': 'Green'...",SOL,[{'address': '3xVDoLaecZwXXtN59o6T3Gfxwjcgf8Hc...,mainnet beta,Okay Bears is a culture shift. A clean collect...,[{'txHash': '5KAybYupDpU9waYsQVMVjrEvdBNYRvRtB...
1,Okay Bear #540,Okay Bears,HR1MLo9sTd3nQHiU4cfVS5VDmNtWV1aC3n46yYkMA9Gt,7ttWYWpc9H9NUQnEaH64qWLwXcHMYwFBu1WrtwjyAHC3,https://bafybeigjdfwmema2xtsnadcdxpshb3m7ar6xg...,"[{'trait_type': 'Background', 'value': 'Green'...",SOL,[{'address': '3xVDoLaecZwXXtN59o6T3Gfxwjcgf8Hc...,mainnet beta,Okay Bears is a culture shift. A clean collect...,[{'txHash': '5wRxy6bDHuesZDLKjMJksPqbHJS7ffx8t...


In [149]:
trait_list = df_traits.traits_trait_type.unique()
assets_traits = df_traits.pivot(
    index="name", columns="traits_trait_type", values="traits_value"
)


In [157]:
assets_traits.to_csv("bear_data.csv")


In [150]:
for t in trait_list:
    i_proportion = assets_traits[t].value_counts(dropna=False, normalize=True)

    assets_traits = assets_traits.merge(
        i_proportion,
        left_on=t,
        right_index=True,
        suffixes=("", "_rarity"),
        how="left",
    )


In [165]:
rarity_cols = [s + "_rarity" for s in trait_list]

rarity = assets_traits[rarity_cols].copy()
rarity["min_rarity"] = rarity[rarity_cols].min(axis=1)
rarity["max_rarity"] = rarity[rarity_cols].max(axis=1)
rarity["mean_rarity"] = rarity[rarity_cols].mean(axis=1)
factorial_rarity = 1 / rarity[rarity_cols]
rarity["factoral_rarity"] = factorial_rarity.sum(axis=1)
rarity = rarity.sort_values("factoral_rarity", ascending=False)
rarity["rarity_rank"] = np.arange(1, rarity.shape[0] + 1, 1)


In [1]:
# rarity.to_csv('okay_bears_rarity.csv')
rarity


NameError: name 'rarity' is not defined